In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
import numpy as np

#### Load card dataset

In [8]:
cards = pd.read_csv("data/cards.csv")

c:\users\vitoj\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3,9,11,14,17,21,24,47,59,60,69,76) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
#cards.columns

In [4]:
def shorten(name):
    if "//" not in name:
        return name
    return name.split("//")[0][:-1]

In [5]:
cards["name_short"] = cards["name"].apply(lambda name: shorten(name))

#### Isolate best drafts

In [10]:
all_drafts = pd.read_csv("draft_data_public.MID.PremierDraft.csv", nrows=100000)

In [11]:
all_drafts.columns

Index(['user_match_win_rate_bucket', 'user_n_matches_bucket', 'draft_id',
       'draft_time', 'expansion', 'event_type', 'user_rank',
       'event_match_wins', 'event_match_losses', 'pack_number',
       ...
       'pool_Vanquish the Horde', 'pool_Vengeful Strangler',
       'pool_Village Watch', 'pool_Vivisection', 'pool_Voldaren Ambusher',
       'pool_Voldaren Stinger', 'pool_Wake to Slaughter', 'pool_Willow Geist',
       'pool_Winterthorn Blessing', 'pool_Wrenn and Seven'],
      dtype='object', length=558)

In [123]:
def remove_bad_drafts(df):
    drafts = df.draft_id.unique()
    suitable_rank = ["platinum", "diamond", "mythic"]
    relevant_draft_ids = []
    for i,draft_id in enumerate(drafts): 
        draft = df[df.draft_id == draft_id].iloc[0]
        #if i % 100 == 0:
        #    print(i)
        wins = draft.event_match_wins
        rank = draft.user_rank
        if wins >= 4 and rank in suitable_rank:
            relevant_draft_ids.append(draft_id)
    relevant_drafts = df[df.draft_id.apply(lambda x: x in relevant_draft_ids)]
    return relevant_drafts

In [124]:
relevant_drafts = remove_bad_drafts(all_drafts)

#### Group colors

In [125]:
relevant_drafts = relevant_drafts.copy()

In [126]:
def find_color(x):
    #print (x)
    return str(cards[cards.name_short == x].iloc[0].colorIdentity)

def find_casting_cost(x):
    #print (x)
    return int(cards[cards.name_short == x].iloc[0].manaValue)

In [127]:
card_list = relevant_drafts[relevant_drafts.columns[272+14:272*2+14]]

In [128]:
g_mask = [1 if "pool_" in c and "G" in  find_color(c.split("_")[1]) else 0 for c in card_list.columns]
b_mask = [1 if "pool_" in c and "B" in  find_color(c.split("_")[1]) else 0 for c in card_list.columns]
u_mask = [1 if "pool_" in c and "U" in  find_color(c.split("_")[1]) else 0 for c in card_list.columns]
r_mask = [1 if "pool_" in c and "R" in  find_color(c.split("_")[1]) else 0 for c in card_list.columns]
w_mask = [1 if "pool_" in c and "W" in  find_color(c.split("_")[1]) else 0 for c in card_list.columns]

In [129]:
relevant_drafts["G"] = np.array(card_list).dot(np.array(g_mask))
relevant_drafts["B"] = np.array(card_list).dot(np.array(b_mask))
relevant_drafts["R"] = np.array(card_list).dot(np.array(r_mask))
relevant_drafts["W"] = np.array(card_list).dot(np.array(w_mask))
relevant_drafts["U"] = np.array(card_list).dot(np.array(u_mask))

In [130]:
one_mask = [1 if "pool_" in c and find_casting_cost(c.split("_")[1]) <= 1 else 0 for c in card_list.columns]
two_mask = [1 if "pool_" in c and find_casting_cost(c.split("_")[1]) == 2 else 0 for c in card_list.columns]
three_mask = [1 if "pool_" in c and find_casting_cost(c.split("_")[1]) == 3 else 0 for c in card_list.columns]
four_mask = [1 if "pool_" in c and find_casting_cost(c.split("_")[1]) == 4 else 0 for c in card_list.columns]
five_mask = [1 if "pool_" in c and find_casting_cost(c.split("_")[1]) == 5 else 0 for c in card_list.columns]
six_mask = [1 if "pool_" in c and find_casting_cost(c.split("_")[1]) >= 6 else 0 for c in card_list.columns]

In [131]:
relevant_drafts["One"] = np.array(card_list).dot(np.array(one_mask))
relevant_drafts["Two"] = np.array(card_list).dot(np.array(two_mask))
relevant_drafts["Three"] = np.array(card_list).dot(np.array(three_mask))
relevant_drafts["Four"] = np.array(card_list).dot(np.array(four_mask))
relevant_drafts["Five"] = np.array(card_list).dot(np.array(five_mask))
relevant_drafts["Six"] = np.array(card_list).dot(np.array(six_mask))

In [49]:
relevant_drafts[relevant_drafts.columns[272+14:-5]].iloc[4].sum()

4

In [34]:
card_list.shape

(2370, 544)

In [35]:
np.array(g_mask).shape

(544,)

In [69]:
relevant_drafts[["One","Two","Three","Four","Five", "Six"]]

,One,Two,Three,Four,Five,Six
1,0,0,0,0,0,0
2,0,0,1,0,0,0
3,0,0,2,0,0,0
4,1,0,2,0,0,0
5,1,0,2,1,0,0
...,...,...,...,...,...,...
9995,2,2,0,2,0,0
9996,2,3,0,2,0,0
9997,2,3,1,2,0,0
9998,3,3,1,2,0,0


In [64]:
relevant_drafts[["G","R","B","W","U"]]

,G,R,B,W,U
1,0,0,0,0,0
2,1,0,0,0,0
3,1,0,0,1,0
4,1,0,0,2,0
5,1,0,0,3,0
...,...,...,...,...,...
9995,0,0,3,2,2
9996,1,0,3,2,3
9997,1,0,3,2,4
9998,1,0,3,3,4


In [29]:
np.array(card_list).dot(np.array(mp))


array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [ ]:
one_draft.pick.apply(lambda x: x+" "+find_color(x)+" "+find_casting_cost(x))

#### Go through data

#### Test and train split

In [132]:
x = relevant_drafts[list(relevant_drafts.columns[14:]) +['pack_number']]
y = relevant_drafts["pick"]

In [133]:
threshold = int(len(x) * 0.8)
x_train = x[:threshold]
x_test = x[threshold:]
y_train = y[:threshold]
y_test = y[threshold:]

In [178]:
clf = RandomForestClassifier(random_state=0, n_estimators=100)
clf.fit(x_train, y_train)

MemoryError: could not allocate 69992448 bytes

In [ ]:
predictions = clf.predict(x_test)

In [ ]:
accuracy_score(predictions, y_test)

In [ ]:
predictions_prob = clf.predict_proba(x_test)

In [ ]:
def check_predictions(predictions, x_test):
    suma = 0
    for i,p in enumerate(predictions):
        column_name = "pack_card_"+p
        suma += x_test.iloc[i][column_name]
    print(suma/len(predictions))
    #print(p)

In [ ]:
check_predictions(predictions, x_test)

#### Change predictions

In [89]:
def parse_pack(row):
    pack = [c for c in row.index if "pack_card_" in c]
    #print(pack)
    names = []
    for i,p in enumerate(pack): 
        if row[p] == 1:
            names.append((i,p.split("_")[2]))
    return names

In [140]:
probs = []
predictions_adjusted = []
for index in range(len(x_test)):
    pack = parse_pack(x_test.iloc[index])
    values = [list(clf.classes_).index(card) for i, card in pack]
    normalized = predictions_prob[index][values] / predictions_prob[index][values].sum()
    pick_index = normalized.argmax()
    pick_card = list(clf.classes_)[values[pick_index]]
    normalized = (normalized*100).astype(int).astype(float)/100 
    predictions_adjusted.append(pick_card)

In [142]:
check_predictions(predictions_adjusted, x_test)

1.0


In [143]:
accuracy_score(predictions_adjusted, y_test)

0.422680412371134

#### Show examples

In [147]:
def shown_pack_predictions(pack_data, probabilities, index, expected, predicted):
    pack = parse_pack(pack_data.iloc[index])
    print(pack_data.iloc[index].pack_number)
    print(expected.iloc[index] +"   //   "+ predicted[index])
    for i,card in pack:
        i = list(clf.classes_).index(card)
        print(card+" "+str(probabilities[index][i]))

In [152]:
len(x_test)

5141

In [176]:
shown_pack_predictions(x_test,predictions_prob,4036, y_test, predictions)

1
Cathar Commando   //   Cathar Commando
Brimstone Vandal 0.03
Cathar Commando 0.15
Harvesttide Sentry 0.06
No Way Out 0.15
Storm Skreelix 0.02
Tavern Ruffian 0.04


In [149]:
display_pack(x_test.iloc[0])

[(139, 'Larder Zombie')]

In [123]:
clf.predict_proba(x_test.iloc[0:2])

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.04, 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  ,
        0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.03, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.01, 0.  , 0.  , 0.  , 0.53, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 

In [120]:
display_pack(x_test.iloc[2])

['Blessed Defiance',
 'Candlegrove Witch',
 'Consider',
 'Drownyard Amalgam',
 'Ecstatic Awakener',
 'Fading Hope',
 'Hound Tamer',
 'Immolation',
 'Lambholt Harrier',
 "Olivia's Midnight Ambush",
 'Otherworldly Gaze',
 "Sigarda's Splendor",
 'Snarling Wolf']

In [90]:
y_test

47117    Tovolar, Dire Overlord
47118          Eccentric Farmer
47119            Skaab Wrangler
47120            Cathartic Pyre
47121         Burn the Accursed
                  ...          
99995          Thraben Exorcism
99996            Smoldering Egg
99997                Immolation
99998               Galedrifter
99999                   Startle
Name: pick, Length: 12851, dtype: object

In [76]:
len(relevant_drafts)

25701

In [73]:
len(df.draft_id.unique())

2579

In [72]:
len(relevant_draft)

668

In [68]:
len)

2579

In [64]:
one_draft.pick.apply(lambda x: x+" "+find_color(x)+" "+find_casting_cost(x))

1           Dawnhart Mentor G 3
2             Borrowed Time W 3
3                Candletrap W 1
4      Search Party Captain W 4
5      Shadowbeast Sighting G 4
6              Bird Admirer G 3
7          Join the Dance G,W 2
8       Celestus Sanctifier W 3
9        Sunrise Cavalier R,W 3
10          Tireless Hauler G 5
11     Path to the Festival G 3
12          Tireless Hauler G 5
13          Stolen Vitality R 2
14        Hedgewitch's Mask W 1
15        Outland Liberator G 2
16         Join the Dance G,W 2
17          Ritual Guardian W 3
18            Bramble Armor G 2
19       Duel for Dominance G 2
20    Might of the Old Ways G 2
21     Dawnhart Rejuvenator G 4
22          Tireless Hauler G 5
23          Lunarch Veteran W 1
24         Return to Nature G 2
25           Firmament Sage U 4
26        Moonrager's Slash R 3
27         Thraben Exorcism W 2
28         Thraben Exorcism W 2
29              Loyal Gryff W 3
30               Clear Shot G 3
31          Cathar Commando W 2
32      

In [161]:
list(df.columns)

['user_match_win_rate_bucket',
 'user_n_matches_bucket',
 'draft_id',
 'draft_time',
 'expansion',
 'event_type',
 'user_rank',
 'event_match_wins',
 'event_match_losses',
 'pack_number',
 'pick_number',
 'pick',
 'pick_maindeck_rate',
 'pick_sideboard_in_rate',
 'pack_card_Abandon the Post',
 'pack_card_Adeline, Resplendent Cathar',
 'pack_card_Ambitious Farmhand',
 'pack_card_Angelfire Ignition',
 'pack_card_Arcane Infusion',
 'pack_card_Ardent Elementalist',
 "pack_card_Arlinn, the Pack's Hope",
 'pack_card_Arrogant Outlaw',
 'pack_card_Augur of Autumn',
 'pack_card_Baithook Angler',
 'pack_card_Baneblade Scoundrel',
 'pack_card_Bat Whisperer',
 'pack_card_Beloved Beggar',
 'pack_card_Bereaved Survivor',
 'pack_card_Bird Admirer',
 'pack_card_Bladebrand',
 'pack_card_Bladestitched Skaab',
 'pack_card_Blessed Defiance',
 'pack_card_Blood Pact',
 'pack_card_Bloodline Culling',
 'pack_card_Bloodthirsty Adversary',
 'pack_card_Bloodtithe Collector',
 'pack_card_Borrowed Time',
 'pack_ca

In [6]:
df.head()

,user_match_win_rate_bucket,user_n_matches_bucket,draft_id,draft_time,expansion,event_type,user_rank,event_match_wins,event_match_losses,pack_number,...,pool_Vanquish the Horde,pool_Vengeful Strangler,pool_Village Watch,pool_Vivisection,pool_Voldaren Ambusher,pool_Voldaren Stinger,pool_Wake to Slaughter,pool_Willow Geist,pool_Winterthorn Blessing,pool_Wrenn and Seven
0,0.64,10,b0d333a808b8448babc0739ca3b24802,2021-09-16 19:28:18,MID,PremierDraft,gold,6,3,0,...,0,0,0,0,0,0,0,0,0,0
1,0.64,10,28ca1deb1aa440218a3a8852bf6c80b8,2021-09-20 15:23:16,MID,PremierDraft,platinum,4,3,0,...,0,0,0,0,0,0,0,0,0,0
2,0.64,10,28ca1deb1aa440218a3a8852bf6c80b8,2021-09-20 15:23:16,MID,PremierDraft,platinum,4,3,0,...,0,0,0,0,0,0,0,0,0,0
3,0.64,10,28ca1deb1aa440218a3a8852bf6c80b8,2021-09-20 15:23:16,MID,PremierDraft,platinum,4,3,0,...,0,0,0,0,0,0,0,0,0,0
4,0.64,10,28ca1deb1aa440218a3a8852bf6c80b8,2021-09-20 15:23:16,MID,PremierDraft,platinum,4,3,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)
clf.predict([[0, 0, 0, 0]]))